# Working with Harmonic Annotations

In [ ]:
import dimcat as dc
import pandas as pd

## Load dataset

Use dimcat's `Dataset` class to load a dataset.
Each dataset consists of several subcorpora (here only `ABC`),
which in turn consist of several pieces (here `n01_op18-1_01`, `n01_op18-1_02`, etc.).

A `Dataset` has several representations of each piece (e.g. a list of chord labels or a list of notes) called *facets*.
Each facet is represented by a dataframe.

Corpora can be processed, e.g. slicing notes according to different criteria (see below).
The output of these operations is again a dataset with facets.

In [ ]:
# this takes some time because it parses the original data, not the preprocessed tsv files
dataset = dc.Dataset()
dataset.load("./ABC", parse_tsv=False, parse_scores=True) # make sure to parse directly from MuseScore files
dataset.data

## Get chord labels

Chord labels are stored in the `expanded` facet.
Using `.get_facet()` returns a single dataframe with all chord labels.
Corpus, piece, and timespan ("interval") are encoded in an hierarchical index.

In [ ]:
labels = dataset.get_facet("expanded")
labels

## Get salami slices

Use the `NoteSlicer` to obtain a sliced version of the dataset.
Querying the note facet returns the sliced notes.

In [ ]:
# this takes some time
salami_dts = dc.NoteSlicer().process_data(dataset)
salami_notes = salami_dts.get_facet("notes")
salami_notes

# Match salami slices with chord labels

Each chord label has an `interval` index that encodes its timespan.
We can use this to find the corresponding slices from the previous step.

Let's try this for a single chord. Start by getting the interval of the first chord in the first piece:

In [ ]:
# zoom in on the chords in one piece
chords = labels.loc[('ABC', 'n01op18-1_01')]
chords

In [ ]:
# get the interval of the first chord...
chord0_interval = chords.index[0]
chord0_interval # this is a pandas Interval

In [ ]:
# and the chord itself
chord0 = chords.loc[chord0_interval]
chord0

Finally, find all slices in the same piece that overlap with the chord:

In [ ]:
salamis = salami_notes.loc[("ABC", "n01op18-1_01")]
salamis[salamis.index.get_level_values(0).overlaps(chord0_interval)]

## Rest...

In [ ]:
salami_notes.loc[("ABC", "n01op18-1_01")]

In [ ]:
salami_notes.loc[("ABC", "n01op18-1_01", pd.Interval(12.0,13.0,closed='left'))]

In [ ]:
str(pd.Interval(0.0,1.0,closed='left'))

In [ ]:
salami_dts.get_slice_info()